In [1]:
import time
import pandas as pd
import undetected_chromedriver as uc

In [2]:
# Call the dataset
acl = pd.read_excel("./query-links/ACL_Query_Links.xlsx")

In [ ]:
# Get the paper links
list_source = []
list_querystring = []
list_querylinks = []
list_paperlinks = []
for index in range(0,len(acl["Query Links from All Attribute"])):
    driver = uc.Chrome()
    time.sleep(5)
    driver.get(acl["Query Links from All Attribute"][index])
    time.sleep(5)
    try:
        # Get links from the first page
        hrefs = driver.find_elements_by_xpath("//*[@class='gs-title']//*[@href]")
        for a in hrefs:
            link = a.get_attribute('href')
            list_source.append("ACL")
            list_querystring.append(acl["Query"][index])
            list_querylinks.append(acl["Query Links from All Attribute"][index])
            list_paperlinks.append(link)

        # Get links from the next page
        for i in range(2,11): # The maximum number of pages we get from ACL search result are 10
            try:
              page_number = "[aria-label='Page "+str(i)+"'"
              driver.find_element_by_css_selector(page_number).click()
              time.sleep(5)
              hrefs = driver.find_elements_by_xpath("//*[@class='gs-title']//*[@href]")
              for a in hrefs:
                link = a.get_attribute('href')
                list_source.append("ACL")
                list_querystring.append(acl["Query"][index])
                list_querylinks.append(acl["Query Links from All Attribute"][index])
                list_paperlinks.append(link)
            except:
              break
    except:
        list_source.append("ACL")
        list_querystring.append(acl["Query"][index])
        list_querylinks.append(acl["Query Links from All Attribute"][index])
        list_paperlinks.append("no_result")
    driver.quit()

In [82]:
# Save to xlsx
acl_paperlinks = pd.DataFrame({'Source': list_source,'Query': list_querystring, 'Query Links': list_querylinks, 'Paper Links': list_paperlinks})
acl_paperlinks.to_excel("./paper-links/PaperLinks_ACL_AllDuplicated.xlsx", index = False)

In [ ]:
# Drop Duplicate
acl_dd = acl_paperlinks.drop_duplicates(['Paper Links'], keep='first')
acl_dd.to_excel("./paper-links/PaperLinks_ACL.xlsx", index = False)